## Ensemble Learning
**Ensemble learning** is a machine learning technique that combines multiple models to create a more powerful and accurate model. It works by training multiple models using different algorithms or different sets of input data, then combining the predictions of these models to create a consensus prediction. This approach can be used to improve the accuracy of a model and reduce the risk of overfitting. Ensemble learning is commonly used in a variety of applications such as image recognition, fraud detection and medical diagnosis.

**Bagging and boosting** are two popular techniques used in ensemble learning. Bagging involves training multiple models on different subsets of the data and combining their predictions to create a consensus prediction. Boosting involves training multiple models sequentially, where each model is trained to correct the mistakes of the previous model. Both techniques can be used to improve the accuracy of a model by reducing the variance or bias.

In [1]:
# So first let's import the dataset. Here we use Diabetes dataset:
import pandas as pd
df = pd.read_csv("diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
df.shape

(768, 9)

In [5]:
# Let's do some explorations. First we need to check the null values:
df.isnull().sum()
#df.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [7]:
# Second is to describe the dataset to see the basic statistics for each column:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [8]:
# So the dataset is looking normal, there is no need for outlier removal or other preprocessing steps. The next thing we 
# want to define is the imbalance in the dataset:
df.Outcome.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [9]:
# So we see that there are 500 samples which says no diabetes and 268 samples which says there are diabetes. So if we look 
# at the ratio:
268/500

0.536

In [10]:
# We see that there is no big imbalance, it's almost 1/2 ratio (it's not 1/100 or 1/1000 ratio), so we go a head and create 
# 'x' and 'y':
x = df.drop(["Outcome"], axis = "columns")
y = df.Outcome

In [14]:
# The next step is to do scalling, because the values are in different range. Here we use standard scalling, you can use 
# Max - Min scalling:
from sklearn.preprocessing import StandardScaler

scl = StandardScaler()
x_scaled = scl.fit_transform(x)
x_scaled[:5]

array([[ 0.63994726,  0.84832379,  0.14964075,  0.90726993, -0.69289057,
         0.20401277,  0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575,  0.53090156, -0.69289057,
        -0.68442195, -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, -1.28821221, -0.69289057,
        -1.10325546,  0.60439732, -0.10558415],
       [-0.84488505, -0.99820778, -0.16054575,  0.15453319,  0.12330164,
        -0.49404308, -0.92076261, -1.04154944],
       [-1.14185152,  0.5040552 , -1.50468724,  0.90726993,  0.76583594,
         1.4097456 ,  5.4849091 , -0.0204964 ]])

In [16]:
# Next we call train_test_split method to split the dataset:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, stratify = y, random_state = 20)

In [17]:
# Train samples are:
x_train.shape

(576, 8)

In [18]:
# Test samples are:
x_test.shape

(192, 8)

In [25]:
# Here for model training we use Decision Tree Classifier, because it's imbalance classifier, it can overfit and it will
# generage high variance model. Additionaly we use cross validation scores right here.
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

scores = cross_val_score(DecisionTreeClassifier(), x, y, cv = 5)
scores

array([0.69480519, 0.66233766, 0.7012987 , 0.78431373, 0.7254902 ])

In [26]:
# to see the average scores:
scores.mean()

0.7136490960020372

In [27]:
# Now let's use bagging classifier:
from sklearn.ensemble import BaggingClassifier

# Here some of the parameters are assigned, you can check all the parameters, and see which give you the better result.
bag_model = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)
bag_model.fit(x_train, y_train)
bag_model.oob_score_

0.7517361111111112

In [28]:
# Now to check the regular scores:
bag_model.score(x_test, y_test)

0.7864583333333334

* So we see the improvment from standalone classifier which has 71% accuracy to 78% accuracty.

In [29]:
# So to use cross validation with the bagging:
bag_model = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)
scores = cross_val_score(bag_model, x, y, cv=5)
scores

array([0.75324675, 0.72727273, 0.74675325, 0.82352941, 0.73856209])

In [30]:
# To see the average scores:
scores.mean()

0.7578728461081402

In [32]:
# Random Forest Classifier also use bagging, it will not sample your data based on rows but it will sample your features (columns) as well.
# To see the performance of Random Forest classifier, let's use it:
from sklearn.ensemble import RandomForestClassifier

scores = cross_val_score(RandomForestClassifier(n_estimators=50), x, y, cv=5)
scores.mean()

0.7656990068754774

* So RF is performing a bit better than DT.

### Exercise
Download heart disease dataset heart.csv in Exercise folder and do following, (credits of dataset: https://www.kaggle.com/fedesoriano/heart-failure-prediction)

   1. Load heart disease dataset in pandas dataframe
   2. Remove outliers using Z score. Usual guideline is to remove anything that has Z score > 3 formula or Z score < -3
   3. Convert text columns to numbers using label encoding and one hot encoding
   4. Apply scaling
   5. Build a classification model using support vector machine. Use standalone model as well as Bagging model and check if you see any difference in the performance.
   6. Now use decision tree classifier. Use standalone model as well as Bagging and check if you notice any difference in performance
   7. Comparing performance of svm and decision tree classifier figure out where it makes most sense to use bagging and why. Use internet to figure out in what conditions bagging works the best.

In [46]:
# Let's first import the dataset:
dfe = pd.read_csv("heart.csv")
dfe.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [47]:
# So first let's check the Null values:
dfe.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [48]:
# There is no null values. Next to check statistics of the dataset:
dfe.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [49]:
# To see imbalance in the dataset:
dfe.HeartDisease.value_counts()

1    508
0    410
Name: HeartDisease, dtype: int64

In [50]:
# To check the ratio:
410/508

0.8070866141732284

In [57]:
# The next step is to convert the string columns to numbers using get_dummies method:
new_dfe = pd.get_dummies(dfe, drop_first=True)
new_dfe.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,0,1,1,0,0,1,0,0,0,1
1,49,160,180,0,156,1.0,1,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0.0,0,1,1,0,0,0,1,0,0,1
3,48,138,214,0,108,1.5,1,0,0,0,0,1,0,1,1,0
4,54,150,195,0,122,0.0,0,1,0,1,0,1,0,0,0,1


In [58]:
# Now let's create 'x' and 'y':
X = new_dfe.drop(["HeartDisease"], axis = "columns")
Y = new_dfe.HeartDisease

In [59]:
# Next step is scall the columns using Standard Scaller:
scle = StandardScaler()
xe_scaled = scle.fit_transform(X)
xe_scaled[:5]

array([[-1.4331398 ,  0.41090889,  0.82507026, -0.55134134,  1.38292822,
        -0.83243239,  0.51595242,  2.07517671, -0.53283777, -0.22967867,
         0.81427482, -0.49044933, -0.8235563 , -1.00218103,  1.15067399],
       [-0.47848359,  1.49175234, -0.17196105, -0.55134134,  0.75415714,
         0.10566353, -1.93816322, -0.48188667,  1.87674385, -0.22967867,
         0.81427482, -0.49044933, -0.8235563 ,  0.99782372, -0.86905588],
       [-1.75135854, -0.12951283,  0.7701878 , -0.55134134, -1.52513802,
        -0.83243239,  0.51595242,  2.07517671, -0.53283777, -0.22967867,
        -1.22808661,  2.03894663, -0.8235563 , -1.00218103,  1.15067399],
       [-0.5845565 ,  0.30282455,  0.13903954, -0.55134134, -1.13215609,
         0.57471149, -1.93816322, -0.48188667, -0.53283777, -0.22967867,
         0.81427482, -0.49044933,  1.21424608,  0.99782372, -0.86905588],
       [ 0.05188098,  0.95133062, -0.0347549 , -0.55134134, -0.5819814 ,
        -0.83243239,  0.51595242, -0.48188667, 

In [63]:
# Next we call train_test_split method to split the dataset:
xe_train, xe_test, ye_train, ye_test = train_test_split(xe_scaled, Y, stratify = Y, random_state = 20)

In [64]:
# Now let's create SVM as stand alone model:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

scores_e = cross_val_score(SVC(), X, Y, cv = 5)
scores_e

array([0.61413043, 0.79347826, 0.69021739, 0.71584699, 0.61748634])

In [65]:
# To average the result:
scores_e.mean()

0.686231884057971

In [66]:
# Now let's create bagging of SVM:
bag_model_e = BaggingClassifier(
    base_estimator=SVC(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)
bag_model_e.fit(xe_train, ye_train)
bag_model_e.oob_score_

0.8837209302325582

In [77]:
# Let's see the actual scores:
bag_model_e.score(xe_test, ye_test)

0.8478260869565217

In [71]:
# Next let's use cross validation scores with bagging:
bag_model_ec = BaggingClassifier(
    base_estimator=SVC(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)
scores_ec = cross_val_score(bag_model_ec, X, Y, cv=5)
scores_ec

array([0.61956522, 0.79347826, 0.69565217, 0.71584699, 0.61748634])

In [72]:
scores_ec.mean()

0.6884057971014493

As you can see above, using bagging in case of SVM doesn't make much difference in terms of model accuracy. Bagging is effective when we have high variance and instable model such as decision tree. Let's explore how bagging changes the performance for a decision tree classifier.

In [73]:
# Now let's use Decision Tree classifier as standalone:
scores_dt = cross_val_score(DecisionTreeClassifier(), X, Y, cv = 5)
scores_dt

array([0.75      , 0.72282609, 0.78804348, 0.67759563, 0.69398907])

In [74]:
scores_dt.mean()

0.7264908529341886

In [79]:
# Next let's use cross validation scores with bagging:
bag_model_dt = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)
bag_model_dt.fit(xe_train, ye_train)
bag_model_dt.oob_score_

0.8531976744186046

In [80]:
# Let's see the actual scores:
bag_model_dt.score(xe_test, ye_test)

0.8304347826086956

In [76]:
# To see the scores with cross validation:
bag_model_dt = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)
scores_dt = cross_val_score(bag_model_dt, X, Y, cv=5)
scores_dt

array([0.8423913 , 0.83152174, 0.82065217, 0.78688525, 0.72677596])

In [81]:
# To see the average:
scores_dt.mean()

0.8016452839154192

* We clearly see the different in both cases (without Cross Vali Sc and with Cross Vali Sc)

#### That's were all about Ensemble (bagging) ...